# PROYECCION 2

import pip
import csv
import operator
import numpy as np
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from networkx.algorithms import bipartite


### CREAR UNA NUEVA RED

G = nx.Graph()


### DATOS

data = pd.ExcelFile('SECOP (PAE) (FINAL).xlsx')
calif = pd.read_excel(data,'NODOS_CALIF',names=["CALIFICACION_CONTRATO"])
rep_contratistas = pd.read_excel(data,'NODOS_REP(C)',names=["REPRESENTANTE (CONTRATISTA)"])

        
### ARCHIVO DE NODOS DE CALIFICACION

for row in list(calif["CALIFICACION_CONTRATO"]):
    G.add_node(row, bipartite=0)
    G.node[row]["NOMBRE (E)"]=row


### ARCHIVO DE NODOS DE REPRESENTANTES DE CONTRATISTAS

for row in list(rep_contratistas["REPRESENTANTE (CONTRATISTA)"]):
    G.add_node(row, bipartite=1)
    G.node[row]["NOMBRE (C)"]=row

### ARCHIVO DE ARCOS DE REPRESENTANTES DE ENTIDADES Y REPRESENTANTES DE CONTRATISTAS

data = pd.ExcelFile('SECOP (PAE) (FINAL).xlsx')
arcos_rep_e_rep_c = pd.read_excel(data,'ARCOS_REP(E)_REP(C)',names=["URL","NOMBRE_REPRESENTATE_ENTIDAD","NOMBRE_REPRESENTANTE","TIPO_PROCESO","OBJETO_CONTRATO","TIPO_CONTRATO","CLASE PLAZO","CLASE ADICION T","CLASE VALOR","CLASE ADICION $","CALIFICACION","NODO_CALIFICACION"])

for index,row in arcos_rep_e_rep_c.iterrows():
    if (row["NODO_CALIFICACION"] in G.nodes()) and (row["NOMBRE_REPRESENTANTE"] in G.nodes()):
        G.add_edge(row["NODO_CALIFICACION"],row["NOMBRE_REPRESENTANTE"])

### TOP AND BOTTOM NODES

top_nodes = set(n for n,d in G.nodes(data=True) if d["bipartite"]==0)
bottom_nodes = set(G)-top_nodes

### FUNCION DE PESO

def peso(G,u,v,weight="CALIFICACION"):
    for iterador in set(G[u]) & set(G[v]):
        w=int(G[u][iterador].get(weight,1))+int(G[v][iterador].get(weight,1))
        if w == 0:
            k=1
        elif w==1:
            k=2
        elif w==2:
            k=3
    return k

### PROYECCION BIPARTITA PARA REPRESENTANTES DE CONTRATISTAS

G_1 = bipartite.generic_weighted_projected_graph(G, bottom_nodes,weight_function=peso)

### EXPORTAR RED BIPARTITA

nx.write_graphml(G_1,"BIPARTITA_1.graphml")

# PROYECCION 1

In [11]:
import pip
import csv
import operator
import numpy as np
import random as rd
import pandas as pd
import networkx as nx
from scipy import stats
import matplotlib.pyplot as plt
from networkx.algorithms import bipartite


## CREAR UNA NUEVA RED

G = nx.Graph()


## DATOS

data = pd.ExcelFile('SECOP (PAE) (FINAL).xlsx')
rep_entidades = pd.read_excel(data,'NODOS_REP(E)',names=["REPRESENTANTE (ENTIDAD)"])
rep_contratistas = pd.read_excel(data,'NODOS_REP(C)',names=["REPRESENTANTE (CONTRATISTA)"])


## ARCHIVO DE NODOS DE REPRESENTANTES DE ENTIDADES

for row in list(rep_entidades["REPRESENTANTE (ENTIDAD)"]):
    if row not in list(rep_contratistas["REPRESENTANTE (CONTRATISTA)"]):
        G.add_node(row, bipartite=0)
        G.node[row]["NOMBRE (E)"]=row
        

## ARCHIVO DE NODOS DE REPRESENTANTES DE CONTRATISTAS

for row in list(rep_contratistas["REPRESENTANTE (CONTRATISTA)"]):
    if row not in list(rep_entidades["REPRESENTANTE (ENTIDAD)"]):
        G.add_node(row, bipartite=1)
        G.node[row]["NOMBRE (C)"]=row

In [12]:
## ARCHIVO DE ARCOS DE REPRESENTANTES DE ENTIDADES Y REPRESENTANTES DE CONTRATISTAS

data = pd.ExcelFile('SECOP (PAE) (FINAL).xlsx')
arcos_rep_e_rep_c = pd.read_excel(data,'ARCOS_REP(E)_REP(C)',names=["URL","NOMBRE_REPRESENTATE_ENTIDAD","NOMBRE_REPRESENTANTE","TIPO_PROCESO","OBJETO_CONTRATO","TIPO_CONTRATO","CLASE PLAZO","CLASE ADICION T","CLASE VALOR","CLASE ADICION $","CALIFICACION","NODO_CALIFICACION"])

for index,row in arcos_rep_e_rep_c.iterrows():
    if (row["NOMBRE_REPRESENTATE_ENTIDAD"] in G.nodes()) and (row["NOMBRE_REPRESENTANTE"] in G.nodes()):
        if (row["NOMBRE_REPRESENTATE_ENTIDAD"],row["NOMBRE_REPRESENTANTE"]) in G.edges():
            G[row["NOMBRE_REPRESENTATE_ENTIDAD"]][row["NOMBRE_REPRESENTANTE"]]["CONTADOR"]+=1
        else:
            G.add_edge(row["NOMBRE_REPRESENTATE_ENTIDAD"],row["NOMBRE_REPRESENTANTE"])
            G[row["NOMBRE_REPRESENTATE_ENTIDAD"]][row["NOMBRE_REPRESENTANTE"]]["CONTADOR"]=1
            G[row["NOMBRE_REPRESENTATE_ENTIDAD"]][row["NOMBRE_REPRESENTANTE"]]["TIPO_PROCESO"] = row["TIPO_PROCESO"]        
            G[row["NOMBRE_REPRESENTATE_ENTIDAD"]][row["NOMBRE_REPRESENTANTE"]]["OBJETO_CONTRATO"] = row["OBJETO_CONTRATO"]
            G[row["NOMBRE_REPRESENTATE_ENTIDAD"]][row["NOMBRE_REPRESENTANTE"]]["TIPO_CONTRATO"] = row["TIPO_CONTRATO"]
            G[row["NOMBRE_REPRESENTATE_ENTIDAD"]][row["NOMBRE_REPRESENTANTE"]]["CLASE PLAZO"] = row["CLASE PLAZO"]
            G[row["NOMBRE_REPRESENTATE_ENTIDAD"]][row["NOMBRE_REPRESENTANTE"]]["CLASE ADICION T"] = row["CLASE ADICION T"]
            G[row["NOMBRE_REPRESENTATE_ENTIDAD"]][row["NOMBRE_REPRESENTANTE"]]["CLASE VALOR"] = row["CLASE VALOR"]
            G[row["NOMBRE_REPRESENTATE_ENTIDAD"]][row["NOMBRE_REPRESENTANTE"]]["CLASE ADICION $"] = row["CLASE ADICION $"]
            G[row["NOMBRE_REPRESENTATE_ENTIDAD"]][row["NOMBRE_REPRESENTANTE"]]["CALIFICACION"] = row["CALIFICACION"]

In [13]:
## TOP AND BOTTOM NODES

top_nodes = set(n for n,d in G.nodes(data=True) if d["bipartite"]==0)
bottom_nodes = set(G)-top_nodes

In [14]:
## FUNCION DE PESO

def peso(G,u,v,weight="CALIFICACION"):
    for iterador in set(G[u]) & set(G[v]):
        w=int(G[u][iterador].get(weight,1))+int(G[v][iterador].get(weight,1))
        if w == 0:
            k=1
        elif w==1:
            k=2
        elif w==2:
            k=3
    return k

In [15]:
## PROYECCION BIPARTITA PARA REPRESENTANTES DE CONTRATISTAS

G_1 = bipartite.generic_weighted_projected_graph(G, bottom_nodes,weight_function=peso)

In [16]:
## CALIFICACIÓN PROMEDIO

data2 = pd.ExcelFile('SECOP (PAE) (FINAL).xlsx')
calificacion_prom = pd.read_excel(data2,'CALIFICACION_PROM',names=["REPRESENTANTE (C)","CALIFICACION"])

In [17]:
representantes_c=calificacion_prom.copy()
representantes_c=representantes_c[["REPRESENTANTE (C)"]]
representantes_c.columns = representantes_c.columns.get_level_values(0)
representantes_c=representantes_c.drop_duplicates(subset=["REPRESENTANTE (C)"])
representantes_c["CALIFICACION_PROM"]=""

for index,row in representantes_c.iterrows():
    lista_row=[]
    for index_2,row_2 in calificacion_prom.iterrows():
        if row["REPRESENTANTE (C)"]==row_2["REPRESENTANTE (C)"]:
            lista_row.append(row_2["CALIFICACION"])
    promedio=np.average(lista_row)
    row["CALIFICACION_PROM"]=promedio

writer = pd.ExcelWriter('CALIFICACION_PROM_REP (C).xlsx')
representantes_c.to_excel(writer,'CALIFICACION',index=False)
writer.save()

### EXPORTAR RED BIPARTITA

nx.write_graphml(G_1,"wawa.graphml")

# RESULTADOS PROYECCIONES

PROYECCION REPRESENTANTE CONTRATISTA_REPRESENTANTE ENTIDAD

lista_nodos_encontrados=["luis gonzalo giraldo marin","nestor ricardo rodriguez ardila","jaime andres caycedo jimenez","carlos julio cristancho morales","janeth guzman corredor","angelica maria leguizamon ramirez","claudia janneth leon velasco","cindy zuleyda lopera baracaldo","paola alejandra carvajal sandoval","yadira ximena moreno sarmiento","leidy jhoanna gallo aristizabal","sandra yolima santamaria guerrero","diego andres rodriguez leiva","yenny liliana martinez vargas","hector enrique moreno celis","david jose zambrano bagarozza","monica yulieth cano medina","laura melissa vallejo diaz","jairo alexander cancino arteaga","lezvia maria de los santos perez fuentes","carol rolay hernandez parada","hernando aguirre tejada","blanca fanny ramirez guerrero","jorge enrique restrepo mantilla","constanza medina puentes","diana paola alonso gutierrez","lina clemencia hernandez lopez","luisa fernanda tellez jimenez","lily andrea collazos fajardo","jeimmy johanna cruz albarracin","jaime alberto pabon perez","enid ramos bubu","doricela diazgranados contreras","pedro luis bohorquez ramirez","antonio manuel parodys carmona","roberto carlos amor olaya","luz aleider herrera algeciras","jaime lasso campo","luis guillermo palacio torres","elbert gildardo rodriguez contreras","saul bailarin rubiano","edelina perez torrente","jose humberto canon dominguez","diana maria villamizar fonseca","claudia andrea roberto shilito","ana maria alzate arismendy","julian andres aramendiz marin","alexander monsalve garciaa","luis hernando santos nino","carolina martinez villa","jose de jesus gonzalez joven","cristian alberto garcia villamizar","lizeth camila vasquez diaz","cielo hipatia rangel luengas","lorna lizbeth marin ariza","luisa fernanda sanchez parra","ariel ivan ruiz parra","jair enrique fuentes correa","viviana rocio nunez mesa","lida rodriguez guiza","farid prado campo","elvis alexander gil zamudio","luis fernando garcia zapata","sebastian gomez lotero","alexander lozano mahecha","sandra patricia torres arteaga","marisol tovar nunez","guillermo andres marin rubio","yamile astrid casas lopez","orlando arnulfo chacon barliza","oscar javier sierra rincon","jose vicente azuero gonzalez","angela alexandra cortes romero","allison andrea paez rodriguez","hector javier venegas salas","jairo alberto silva ceballos","alexander monsalve garcia","ivan dario rodriguez bustamante","ana catalina suarez galvis","maria fernanda bohorquez torres","camilo andres garcia ortiz","yineth cruz barbosa","natalia rodriguez corzo","geluy alvarez calderon","ana maria salazar molano"]

df_degree=pd.DataFrame(columns=["NOMBRE","GRADO"])
for i in lista_nodos_encontrados:
    df_degree=df_degree.append(pd.DataFrame({'NOMBRE': i, 'GRADO': G_1.degree(i)}, index=[0]))
    df_degree=df_degree.drop_duplicates()

df_degree=df_degree[["NOMBRE","GRADO"]]
df_degree.to_csv("df_degree.csv",sep="/",index=False)

df_fuerza=pd.DataFrame(columns=["NOMBRE","FUERZA"])
for i in lista_nodos_encontrados:
    weight=0
    list_weight=[]
    for j in G_1.nodes():
        if i != j:
            try:
                weight = G_1[i][j].get("weight")
                list_weight.append(weight)
            except:
                pass
    
    df_fuerza=df_fuerza.append(pd.DataFrame({'NOMBRE': i, 'FUERZA': np.average(list_weight)}, index=[0]))


df_fuerza=df_fuerza.drop_duplicates()    
df_fuerza=df_fuerza[["NOMBRE","FUERZA"]]
df_fuerza.to_csv("df_fuerza.csv",sep="/",index=False)

df_betweenn=pd.DataFrame(columns=["NOMBRE","BETWEENN"])
betweenness = nx.betweenness_centrality(G_1)
for i in lista_nodos_encontrados:
    df_betweenn=df_betweenn.append(pd.DataFrame({'NOMBRE': i, 'BETWEENN': betweenness[i]}, index=[0]))
    df_betweenn=df_betweenn.drop_duplicates()

df_betweenn=df_betweenn[["NOMBRE","BETWEENN"]]
df_betweenn.to_csv("df_betweenn.csv",sep="/",index=False)

df_clustering=pd.DataFrame(columns=["NOMBRE","CLUSTERING"])
clustering = nx.clustering(G_1)
for i in lista_nodos_encontrados:
    df_clustering=df_clustering.append(pd.DataFrame({'NOMBRE': i, 'CLUSTERING': clustering[i]}, index=[0]))
    df_clustering=df_clustering.drop_duplicates()

df_clustering=df_clustering[["NOMBRE","CLUSTERING"]]
df_clustering.to_csv("df_clustering.csv",sep="/",index=False)

df_closeness=pd.DataFrame(columns=["NOMBRE","CLOSENESS"])
closeness = nx.closeness_centrality(G_1)
for i in lista_nodos_encontrados:
    df_closeness=df_closeness.append(pd.DataFrame({'NOMBRE': i, 'CLOSENESS': closeness[i]}, index=[0]))
    df_closeness=df_closeness.drop_duplicates()

df_closeness=df_closeness[["NOMBRE","CLOSENESS"]]
df_closeness.to_csv("df_closeness.csv",sep="/",index=False)

# ANALISIS DE HOMOFILIA

In [18]:
#HOMOFILIA

Prop = dict()

A = 0
B = 0

Nodes_Cal=set()


#BASE DE DATOS DE CALIFICACIONES

data3 = pd.ExcelFile('CALIFICACION_PROM_REP (C).xlsx')
nodos_calificacion_rango = pd.read_excel(data3,'CALIFICACION',names=["REPRESENTANTE (C)","CALIFICACION_PROM","CALIFICACION_PROM_VALOR","CALIFICACION_RANGO"])


#AGREGAR ATRIBUTO A CADA UNO DE LOS NODOS

for index,row in nodos_calificacion_rango.iterrows():
    for n in G_1.nodes():
        if row["REPRESENTANTE (C)"]==n:
            G_1.node[n]["RANGO"]=row["CALIFICACION_RANGO"]
            G_1.node[n]["ANORM"]=row["CALIFICACION_PROM_VALOR"]

            
#PROPORCIONES

for n in G_1.nodes():
    if G_1.node[n]['RANGO']=='A':
        A=A+1
    elif G_1.node[n]['RANGO']=='B':
        B=B+1

Prop["A"]=float(float(A)/float(G_1.number_of_nodes()))
Prop["B"]=float(float(B)/float(G_1.number_of_nodes()))


#GRADO POR RANGO EN LA RED

Grado_G_1 = nx.degree(G_1)

Grado_R_A=list()
Grado_R_B=list()

for n in G_1.nodes():
    if G_1.node[n]['RANGO']=='A':
        Grado_R_A.append(Grado_G_1[n])
    elif G_1.node[n]['RANGO']=='B':
        Grado_R_B.append(Grado_G_1[n])


#Calculando la cantidad de arcos para combinación entre RANGOos:
O = dict()

O['A-A']=0
O['B-A']=0
O['B-B']=0

for e in G_1.edges():
    Source = e[0]
    Target = e[1]
    if (G_1.node[Source]['RANGO']=='A') and (G_1.node[Target]['RANGO']=='A'):
        O['A-A']+=1
    elif G_1.node[Source]['RANGO']!=G_1.node[Target]['RANGO']:
        O['B-A']+=1
    elif (G_1.node[Source]['RANGO']=='B') and (G_1.node[Target]['RANGO']=='B'):
        O['B-B']+=1
        

#LISTAS DE RESULTADOS DE PERMUTACIONES
r = dict()

r['A-A']=list()
r['B-A']=list()
r['B-B']=list()

In [ ]:
for i in range(4000):
    #CREAR GRAFO DEL MODELO NULO:
    
    G_nulo = G_1
    for n in G_nulo.nodes():
        if rd.random() < Prop["A"]:
            G_nulo.node[n]['RANGO'] = 'A'
        else:
            G_nulo.node[n]['RANGO'] = 'B'
    
    #GRADO DEL MODELO NULO POR RANGO PARA LA APROBACION CONTRAFACTUAL DE LA RED
    
    Grado_Null = nx.degree(G_nulo)
    Grado_R_A_Null=list()
    Grado_R_B_Null=list()

    for n in G_nulo.nodes():
        if G_nulo.node[n]['RANGO']=='A':
            Grado_R_A_Null.append(Grado_Null[n])
        elif G_nulo.node[n]['RANGO']=='B':
            Grado_R_B_Null.append(Grado_Null[n])

    #EVALUAR EL PORCENTAJE DE ACEPTACION DE LA RED GENERADA
    
    if (abs((np.mean(Grado_R_A)/np.mean(Grado_R_A_Null))-1) < 0.1) and (abs((np.mean(Grado_R_B)/np.mean(Grado_R_B_Null))-1) < 0.1):
        
        #Calculando la cantidad de arcos para combinación entre sexos:
        
        S = dict()
        S['A-A']=0
        S['B-A']=0
        S['B-B']=0
        for e in G_1.edges():
            Source = e[0]
            Target = e[1]
            if (G_nulo.node[Source]['RANGO']=='A') and (G_nulo.node[Target]['RANGO']=='A'):
                S['A-A']+=1
            elif G_nulo.node[Source]['RANGO'] != G_nulo.node[Target]['RANGO']:
                S['B-A']+=1
            elif (G_nulo.node[Source]['RANGO']=='B') and (G_nulo.node[Target]['RANGO']=='B'):
                S['B-B']+=1
        
        #RADIO OBSERVADO Y PERMUTACION
        
        r['A-A'].append(float(O["A-A"])/float(S['A-A']))
        r['B-A'].append(float(O["B-A"])/float(S['B-A']))
        r['B-B'].append(float(O["B-B"])/float(S['B-B']))

In [ ]:
#PARAMETROS DE LA DISTRIBUCION: MEDIA, VARIANZA

mu = 1
sigma = 0.1


#GENERACION DE DATOS

y=r["A-A"]


#GENERACION DEL HISTOGRAMA

count, bins, ignored =plt.hist(y,100,normed=True)


#GENERACION DE LA FUNCION DE DENSIDAD DE PROBABILIDAD CORRESPONDIENTE

plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (bins - mu)**2 / (2 * sigma**2) ),linewidth=2, color='r')

plt.show()

print('normaltest teststat = %6.3f pvalue = %6.4f' % stats.normaltest(y))
print "Verificación de homofilia para el tipo de conexión [A-A]"

#GENERACION DE LOS DATOS

y=r["B-B"]


#GENERACION DEL HISTOGRAMA

count, bins, ignored =plt.hist(y,100,normed=True)


#GENERACION DE LA FUNCION DE DENSIDAD DE PROBABILIDAD CORRESPONDIENTE

plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (bins - mu)**2 / (2 * sigma**2) ), linewidth=2, color='r')

plt.show()

print('normaltest teststat = %6.3f pvalue = %6.4f' % stats.normaltest(y))
print "Verificación de homofilia para el tipo de conexión [B-B]"


#GENERACION DE LOS DATOS

y=r["B-A"]


#GENERACION DEL HISTOGRAMA

count, bins, ignored =plt.hist(y,100,normed=True)


#GENERACION DE LA FUNCION DE DENSIDAD DE PROBABILIDAD CORRESPONDIENTE

plt.plot(bins, 1/(sigma * np.sqrt(2 * np.pi)) * np.exp( - (bins - mu)**2 / (2 * sigma**2) ), linewidth=2, color='r')

plt.show()

print('normaltest teststat = %6.3f pvalue = %6.4f' % stats.normaltest(y))
print "Verificación de heterofilia para el tipo de conexión [B-A]"